This notebook generates pharmacophore features from pretrained model (Used for Docking Score prediction - Alpha)

In [2]:
import sys
import os
import torch
from tqdm import tqdm

sys.path.append("..")

/home/tsa87/anaconda3/envs/tacogfn/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from src.scoring import PrecalculationModule

In [5]:
MODEL_PATH = '../model_weights/model.tar'
HEAD_PATH = '../model_weights/20240117_500.pth'

split_file = torch.load('../dataset/pocket_to_ligands.pt')

In [4]:
predictor = PrecalculationModule(MODEL_PATH, HEAD_PATH, 'cuda')

In [5]:
failed_pairs = []
i = 0
for pdb_id, lig_path in tqdm(split_file.items()):
    lig_path = os.path.join('../dataset/crossdocked_pocket10/', lig_path)
    rec_path = os.path.join('../dataset/crossdock/', pdb_id + '_rec.pdb')
    out_path = rec_path.replace('crossdock', 'docking_pharmacophores').replace('.pdb', '.pt')   
    
    if os.path.exists('out_path'):
        continue
    try:
        cache = predictor.run(rec_path, ref_ligand_path=lig_path)
        torch.save(cache, out_path)
    except Exception as e:
        failed_pairs.append((rec_path, lig_path))
        print(e)

  0%|          | 0/15307 [00:07<?, ?it/s]


Check if all files are saved correctly (not corrupted)

In [6]:
for pdb_id, lig_path in tqdm(split_file.items()):
    lig_path = os.path.join('../dataset/crossdocked_pocket10/', lig_path)
    rec_path = os.path.join('../dataset/crossdock/', pdb_id + '_rec.pdb')
    out_path = rec_path.replace('crossdock', 'docking_pharmacophores').replace('.pdb', '.pt')
    
    if os.path.exists(out_path):
        try:
            torch.load(out_path)
        except:
            cache = predictor.run(rec_path, ref_ligand_path=lig_path)
            print('recomputed', out_path)
            torch.save(cache, out_path)

100%|██████████| 15307/15307 [00:10<00:00, 1401.92it/s]
